This notebook compares 2018 aggregates vs 2018 predictions **for metro areas**

In [1]:
import re
import numpy as np
import pandas as pd
from math import sqrt
import geopandas as gpd
import rasterio as rio
from shapely.wkt import loads
from tqdm import tqdm

import sys
sys.path.insert(0, '../utils')
from settings import *
import geoutils
import modelutils

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
def generate_satellite_features(gdf, year = 2018):
    '''
    Generates features derived from satellite images by piercing through rasters using the centroids of the grid from gdf
    
    Args
        gdf (GeoDataFrame): indicator labelled grid
    Returns
        gdf (GeoDataFrame): indicator labelled grid with features
    '''
    # satellite image derived - pierce through rasters
    geom_col = 'centroid_geometry'
    tifs_with_250m = ['nighttime_lights', 'population', 'elevation', 'urban_index']
    satellite_features_ = [f + '_250m' if f in tifs_with_250m else f for f in satellite_features] + ['nearest_highway']
    pois_ = ['waterway', 'commercial', 'restaurant', 'hospital', 'airport']
    poi_features_ = ['clipped_nearest_' + poi for poi in pois_]
    for feature in tqdm(poi_features_ + satellite_features_):
        if feature in satellite_features_:
            tif_file = feats_dir + f'{year}_{area}_{feature}.tif'
        else:
            tif_file = feats_dir + f'2018_{area}_{feature}.tif'
        raster = rio.open(tif_file)

        # Perform point sampling
        pxl = []
        for index, row in gdf.iterrows():
            for val in raster.sample([(row[geom_col].x, row[geom_col].y)]):
                pxl.append(val[0])

        # Add column to geodataframe
        col_name = feature.replace('clipped_','')
        gdf[col_name] = pxl
        
    # remove _250m suffix
    feats_250m = ['nighttime_lights_250m', 'population_250m', 'elevation_250m', 'urban_index_250m']
    gdf.columns = [f[:-5] if f in feats_250m else f for f in gdf.columns]
    
    return gdf

In [3]:
# !gsutil cp gs://immap-wash-training/grid/grids_in_metro_areas_2018.csv {data_dir}
# !gsutil cp gs://immap-wash-training/features/2018_*.tif {feats_dir}
# !gsutil cp gs://immap-wash-training/features/2018_colombia_aridity_cgiarv2.tif {feats_dir}2018_colombia_aridity_cgiarv2.tif
# !gsutil cp gs://immap-wash-training/features/2018_colombia_nearest_highway.tif {feats_dir}2018_colombia_nearest_highway.tif

In [36]:
df = pd.read_csv(data_dir + 'grids_in_metro_areas.csv')
geom_col = 'centroid_geometry'
df[geom_col] = df[geom_col].apply(loads)
gdf = gpd.GeoDataFrame(df, geometry = geom_col)

## Generate data for 2018

In [17]:
# gdf = generate_satellite_features(gdf, year = 2018)
# test_df = geoutils.generate_training_data(gdf)
# cols = ['id', 'metro_id', 'geometry'] + poi_features + satellite_features
# print(test_df.shape)
# print('Complete cases %: ' + str(test_df.dropna(subset = cols).shape[0]/test_df.shape[0]))
# test_df.to_csv(data_dir + '20200924_dataset_2018.csv')
test_df = pd.read_csv(data_dir + '20200924_dataset_2018.csv')
print(test_df.shape)
test_df.head(3)

(26542, 42)


,Unnamed: 0,pixelated_urban_area_id,id,geometry,adm1_name,adm2_name,centroid_geometry,metro_id,nearest_waterway,nearest_commercial,...,lag_aridity_cgiarv2,lag_temperature,lag_nighttime_lights,lag_population,lag_elevation,lag_urban_index,lag_nearest_highway,nighttime_lights_area_mean,x,y
0,0,NaN,18290623,"POLYGON ((-76.4798939023438 3.4579949602661, -...",Valle del Cauca,Cali,POINT (-76.4787677773021 3.45686883565232),9,397.931305,2778.615723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.478768,3.456869
1,1,NaN,17961848,"POLYGON ((-76.5744884023438 3.4557427102661, -...",Valle del Cauca,Cali,POINT (-76.57336227725401 3.4546165853639),9,296.926178,3046.067139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.573362,3.454617
2,2,NaN,18110633,"POLYGON ((-76.5316956523438 3.3363734602661, -...",Valle del Cauca,Cali,POINT (-76.53056952747269 3.33524733538718),9,1662.000244,670.668213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.530570,3.335247


## Train full model on 2018

In [8]:
train_df = pd.read_csv(data_dir + '20200830_dataset.csv')
print(train_df.shape)

(57143, 45)


In [9]:
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

def model(train_df, test_df):
    global clf
    clf = RandomForestRegressor(random_state=42)
    
    feats = []
    for indicator in tqdm(indicators):

        avg_metrics = {'correlation':[], 'r2':[], 'mae':[], 'rmse':[]}
        X_train, y_train = train_df[features], train_df[indicator]
        X_test = test_df[features]
        scaler = RobustScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        clf = joblib.load(model_dir + 'model_' + indicator + '_2018_250mv2.pkl')
        # clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        test_df['pred_' + indicator] = y_pred
        
        feature_importances = pd.DataFrame({'feature': list(train_df[features].columns)
                                            , 'importance': list(clf.feature_importances_)})
        top_features = (feature_importances
                            .sort_values(by=['importance'], ascending = False))
        top_features['indicator'] = indicator
        feats.append(top_features)
        
#         joblib.dump(clf, model_dir + 'model_' + indicator + '_2018_250mv2.pkl')
    
#     joblib.dump(scaler, scaler_dir + 'scaler_2018_250mv2.pkl')
    
    return test_df, pd.concat(feats, axis = 0).reset_index(drop = True)

In [18]:
test_df.shape

(26542, 42)

In [28]:
test_df.dropna(subset = ['population']).shape

(26533, 42)

In [29]:
df = test_df.copy()
df['population'] = df['population'].fillna(0)
df = df.dropna(subset = ['population']).reset_index(drop = True)
test_df = df.copy()
print(test_df.shape)

(26542, 42)


In [33]:
test_df, top_features = model(train_df, test_df)
# top_features.to_csv('top_features_2018.csv', index = False)
test_df.to_csv(data_dir + '20200924_predictions2018.csv', index = False)

## Aggregate grid predictions to metro areas

In [43]:
raw = pd.read_csv(data_dir + '20200924_predictions2018.csv')
test_df = pd.merge(raw.drop(labels = ['metro_id'], axis = 1), df[['id', 'metro_id']], how = 'left', on = 'id').dropna(subset = ['metro_id'])

In [45]:
# TODO: column names are quite confusing..
# estimate number of households in grid with wash access
for indicator in indicators:
    test_df['pred_' + indicator.replace('perc_', '')] = test_df['population']*test_df['pred_' + indicator]

# sum household count by area
hh_cols = ['pred_' + ind.replace('perc_', '') for ind in indicators]
pred_metro = (test_df[['metro_id', 'population'] + hh_cols]
                    .groupby('metro_id').agg('sum').reset_index())

# calculate new percentage hh no access
for indicator in indicators:
    pred_metro['pred_' + indicator] = 100*pred_metro['pred_' + indicator.replace('perc_', '')] / pred_metro['population']

## Compare to actual values

In [47]:
# !gsutil cp gs://immap-wash-training/indicators/20200908_GEIH_Metro_Areas_2018.csv {data_dir}
true_metro = pd.read_csv(data_dir + '20200908_GEIH_Metro_Areas_2020.csv')

spanish = {
    'personas': 'population',
    'c_acueduct': 'hh_no_water_supply',
    'c_alcantar': 'hh_no_sewage',
    'c_sanitari': 'hh_no_toilet',
    'mc_acueduc': 'perc_hh_no_water_supply',
    'mc_alcanta': 'perc_hh_no_sewage',
    'mc_sanitar': 'perc_hh_no_toilet',
}

pd.set_option('display.float_format', lambda x: '%.6f' % x)

df1 = pred_metro.sort_values('metro_id', ascending = True)

df2 = true_metro[['OBJECTID', 'geometry'] + list(spanish.keys())].rename(columns=spanish)

cons = pd.merge(
    df1,#[['metro_id', 'pred_perc_hh_no_water_supply', 'pred_perc_hh_no_sewage', 'pred_perc_hh_no_toilet']], 
    df2,#[['OBJECTID', 'geometry', 'perc_hh_no_water_supply', 'perc_hh_no_sewage', 'perc_hh_no_toilet']], 
    left_on = 'metro_id', right_on = 'OBJECTID'
).drop(labels = 'OBJECTID', axis = 1)

cons.to_csv(data_dir + 'metro_area_predictions_2018.csv')

In [49]:
cons.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
metro_id,23.000000,12.000000,6.782330,1.000000,6.500000,12.000000,17.500000,23.000000
population_x,23.000000,104642.905326,207420.738339,3779.139582,25641.732428,33311.747970,68541.491614,977025.830954
pred_hh_no_water_supply,23.000000,3825.131890,5405.677400,432.331110,1248.050135,1721.822054,5218.108991,25640.569841
pred_hh_no_toilet,23.000000,3448.764105,6512.985649,577.204917,983.459131,1160.575000,2395.891185,31077.488512
pred_hh_no_sewage,23.000000,5827.827315,7931.727906,663.294335,1698.465348,2560.425494,6780.920328,36757.762866
pred_perc_hh_no_water_supply,23.000000,7.482862,11.670938,1.776939,3.061296,4.323007,5.390534,58.346067
pred_perc_hh_no_toilet,23.000000,5.068869,6.933614,2.516443,3.085623,3.477317,3.860727,36.584901
pred_perc_hh_no_sewage,23.000000,10.762488,12.430853,2.726228,5.081230,5.960865,11.163652,62.122381
population_y,23.000000,343968.000000,599073.573509,32560.000000,89878.000000,132743.000000,251821.500000,2769346.000000
hh_no_water_supply,23.000000,2999.466329,5444.379756,0.000000,123.989871,629.053780,3014.153150,19516.640000


In [50]:
for indicator in indicators:
    print(indicator)
    print(modelutils.calculate_metrics(cons[indicator], cons['pred_' + indicator]))

perc_hh_no_water_supply
{'correlation': 0.988039691943823, 'r2': 0.9762224328564447, 'mae': 3.6291356603229965, 'rmse': 4.050000846765509}
perc_hh_no_toilet
{'correlation': 0.9924261413805374, 'r2': 0.9849096460954625, 'mae': 3.8616014067553777, 'rmse': 6.1725261127173585}
perc_hh_no_sewage
{'correlation': 0.9810060395646845, 'r2': 0.9623728496623872, 'mae': 5.365618694982769, 'rmse': 6.648911012488556}


In [51]:
# population from world pop (_x) is a subset only of population from GEIH (_y)
cons[['metro_id', 'population_x', 'population_y']]

,metro_id,population_x,population_y
0,1,28097.396715,99866.000000
1,2,24330.108774,121065.000000
2,3,11548.487872,55073.000000
3,4,3779.139582,32560.000000
4,5,13725.477257,66999.000000
5,6,214497.039681,462727.000000
6,7,977025.830954,2769346.000000
7,8,77215.466761,341502.000000
8,9,244232.870972,845629.000000
9,10,66428.015970,258850.000000
